In [ ]:
#bertopic을 통한 토픽모델링

In [7]:
!pip install bertopic

     |████████████████████████████████| 58 kB 2.8 MB/s 
     |████████████████████████████████| 88 kB 4.1 MB/s 
     |████████████████████████████████| 5.2 MB 23.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 79 kB 7.0 MB/s 
     |████████████████████████████████| 4.2 MB 29.7 MB/s 
     |████████████████████████████████| 1.2 MB 47.3 MB/s 
     |████████████████████████████████| 84 kB 2.8 MB/s 
     |████████████████████████████████| 636 kB 45.5 MB/s 
     |████████████████████████████████| 6.6 MB 42.4 MB/s 
     |████████████████████████████████| 1.1 MB 46.9 MB/s 
  Created wheel for hdbscan: filename=hdbscan-0.8.28-cp37-cp37m-linux_x86_64.whl size=2330811 sha256=2645a96ca34ddf514530d0694b34a2471e8c0955e265d7ec22f683b2625562c1
  Stored in directory: /root/.cache/pip/wheels/6e/7a/5e/259ccc841c085fc41b99ef4a71e896b62f5161f2bc8a14c97a
  Created wheel for sentence-

In [8]:
!pip install bertopic[visualization]

In [5]:
# Colab에 Mecab 설치
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 115, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 115 (delta 11), reused 10 (delta 3), pack-reused 91
Receiving objects: 100% (115/115), 1.27 MiB | 17.82 MiB/s, done.
Resolving deltas: 100% (50/50), done.
/content/Mecab-ko-for-Google-Colab/Mecab-ko-for-Google-Colab
Installing konlpy.....
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2022-05-15 20:01:18--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::3403:4be7, 2406:da00:ff00::22cd:e0db, ...
Connecting to bitbucket.org (bitbucket.org)|104.192.141.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://bbuseruploads.s3.amazonaws.com/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0

In [9]:
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Mecab
from bertopic import BERTopic

In [10]:
text_file = "/content/review.txt"

In [12]:
documents = [line.strip() for line in open(text_file, encoding="utf-8").readlines()]

In [13]:
preprocessed_documents = []

for line in tqdm(documents):
  # 빈 문자열이거나 숫자로만 이루어진 줄은 제외
  if line and not line.replace(' ', '').isdecimal():
    preprocessed_documents.append(line)

100%|██████████| 810/810 [00:00<00:00, 304125.53it/s]


In [14]:
class CustomTokenizer:
    def __init__(self, tagger):
        self.tagger = tagger
    def __call__(self, sent):
        sent = sent[:1000000]
        word_tokens = self.tagger.nouns(sent)
        result = [word for word in word_tokens if len(word) > 1]
        return result

In [15]:
custom_tokenizer = CustomTokenizer(Mecab())

In [16]:
vectorizer = CountVectorizer(tokenizer=custom_tokenizer, max_features=3000)

In [17]:
model = BERTopic(embedding_model="sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens", \
                 vectorizer_model=vectorizer,
                 nr_topics=50,
                 top_n_words=10,
                 #language="multilingual",
                 calculate_probabilities=True)

In [ ]:
topics, probs = model.fit_transform(preprocessed_documents)

In [19]:
model.get_topic_info()

,Topic,Count,Name
0,-1,317,-1_카페_커피_친절_분위기
1,0,235,0_커피_음식_가격_사람
2,2,56,2_주차장_조양_여자_화장실
3,8,41,8_가격_음료_카페_대기
4,5,32,5_남자_식겁_컨셉_정서방
5,10,30,10_이걸_맛집_옹이_배탈
6,7,16,7_다음_갈게_부모_친절
7,11,16,11_친절_안락_오브제_바다
8,9,15,9_드라이브_힐링_맛집_분위기
9,4,14,4_조개찜_딸기_허버_수칙


In [20]:
model.visualize_topics()

In [21]:
model.visualize_distribution(probs[0])

In [22]:
for i in range(0, 50):
  print(i,'번째 토픽 :', model.get_topic(i))

0 번째 토픽 : [('커피', 0.03194977520636129), ('음식', 0.03190447760169624), ('가격', 0.028396501242247484), ('사람', 0.026590346320477895), ('강된장', 0.026106830164768287), ('반찬', 0.025926212697245453), ('꽃게', 0.024848069692592945), ('비빔밥', 0.023778016645783235), ('직원', 0.022241339676703702), ('정도', 0.022099887570116078)]
1 번째 토픽 : [('자리', 0.14537311178278509), ('정도', 0.09452732878637017), ('주차장', 0.08465593746277728), ('방송', 0.07738607603756538), ('주변', 0.07273329418090939), ('사람', 0.07266355645438771), ('이상', 0.0674380480338452), ('분위기', 0.059031434720629855), ('탄지', 0.05691590775281888), ('가운데', 0.05691590775281888)]
2 번째 토픽 : [('주차장', 0.03794304173807354), ('조양', 0.03749002047567257), ('여자', 0.03696096788019535), ('화장실', 0.03689709382534073), ('키즈', 0.0359682523873823), ('방직', 0.0359682523873823), ('남자', 0.03483971317129312), ('강된장', 0.033574597863062466), ('맛집', 0.03303411924076639), ('매장', 0.031327221762710536)]
3 번째 토픽 : [('여긴', 0.18466474016521497), ('지인', 0.17450484898955), ('이상', 0.166242